In [11]:
pip install sklearn

    100% |████████████████████████████████| 7.3MB 2.4MB/s ta 0:00:011
  Using cached https://files.pythonhosted.org/packages/dc/29/162476fd44203116e7980cfbd9352eef9db37c49445d1fec35509022f6aa/scipy-1.4.1-cp36-cp36m-manylinux1_x86_64.whl
  Running setup.py install for sklearn ... done
Note: you may need to restart the kernel to use updated packages.


In [13]:
import base64
import numpy as np
import pandas as pd
import nltk
from collections import Counter
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB

Считываем данные

In [17]:
train = pd.read_csv("/home/emin/PycharmProjects/machine_learning/data/spooky author/train.zip")
test = pd.read_csv("/home/emin/PycharmProjects/machine_learning/data/spooky author/test.zip")
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [18]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


Необходимо привести слова к нормальной форме(т.е лемматизация), поэтому переопределим метод в классе LemmaCountVectorizer
Реализуем лемматизатор для каждого списка

In [20]:
lemmatizer = WordNetLemmatizer()
class LemmaCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(LemmaCountVectorizer, self).build_analyzer()
        return lambda doc: (lemmatizer.lemmatize(w) for w in analyzer(doc))

In [21]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/emin/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [22]:
# сохраняем текст в виде списка
text = list(train.text.values)
text_frame_vectorizer = LemmaCountVectorizer(max_df=0.95, min_df=2,stop_words='english',decode_error='ignore')
text_frame = text_frame_vectorizer.fit_transform(text)

In [23]:
len(text)

19579

In [24]:
text[1]

'It never once occurred to me that the fumbling might be a mere mistake.'

In [25]:
f_names = text_frame_vectorizer.get_feature_names()
count_vector = np.asarray(text_frame.sum(axis=0)).ravel()
zip_obj = list(zip(f_names, count_vector))
x, y = (list(x) for x in zip(*sorted(zip_obj, key=lambda x: x[1], reverse=True)))
Y = np.concatenate([y[0:15], y[-16:-1]])
X = np.concatenate([x[0:15], x[-16:-1]])

In [26]:
train['num'] = train.author.map({'EAP':0, 'HPL':1, 'MWS':2})
X = train['text']
y = train['num']
print (X.head())
print (y.head())

0    This process, however, afforded me no means of...
1    It never once occurred to me that the fumbling...
2    In his left hand was a gold snuff box, from wh...
3    How lovely is spring As we looked from Windsor...
4    Finding nothing else, not even gold, the Super...
Name: text, dtype: object
0    0
1    1
2    0
3    2
4    1
Name: num, dtype: int64


In [27]:
train.head()

,id,text,author,num
0,id26305,"This process, however, afforded me no means of...",EAP,0
1,id17569,It never once occurred to me that the fumbling...,HPL,1
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,0
3,id27763,How lovely is spring As we looked from Windsor...,MWS,2
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,1


Нужно разделить данные на обучающий набор и тестовый набор. Тестовый набор-это набор, который будем использовать только для тестирования модели. Поэтому мы разделим его на 70% для обучения и 30% для тестирования.

In [28]:
percent = int(float(0.7)* len(X))
X_train = X[:percent]
X_test = X[percent:]
y_train = y[:percent]
y_test = y[percent:]

In [29]:
X_cv = text_frame_vectorizer.fit_transform(X)
X_train_cv = text_frame_vectorizer.transform(X_train)
X_test_cv = text_frame_vectorizer.transform(X_test)
print (X_train_cv.shape)

(13705, 13781)


Передаем данные в data для обучения и тестирования его на результат.

In [30]:
data = MultinomialNB()
data.fit(X_train_cv, y_train)
data.score(X_test_cv, y_test)

0.8171603677221655

Обновим словарный запас и преобразуем необработанные тестовые данные в векторизованную форму

In [33]:
X_test = text_frame_vectorizer.transform(test["text"])

Помещаем все обучающие данные без какого либо разделения в нашу наивную байесовскую модель, потом даем ей тестовые векторизованные данные для предсказания вероятностей

In [34]:
data = MultinomialNB()
data.fit(X_cv, y)
predict = data.predict_proba(X_test)
predict.shape

(8392, 3)

In [35]:
predict

array([[4.35167111e-03, 6.08155656e-04, 9.95040173e-01],
       [9.99939791e-01, 5.74819399e-05, 2.72705941e-06],
       [6.16386414e-01, 3.83546094e-01, 6.74921258e-05],
       ...,
       [9.98330613e-01, 5.38192133e-04, 1.13119463e-03],
       [3.70662255e-03, 4.21746570e-05, 9.96251203e-01],
       [1.07853729e-02, 9.89212838e-01, 1.78932449e-06]])

Получили результат с 8392 строками, представляющими каждый текст, и 3 столбцами, каждый из которых представляет вероятность каждого автора.

Вывод результата в csv таблицу

In [36]:
result_submission = pd.DataFrame()
result_submission["id"] = test["id"]
result_submission["EAP"] = predict[:,0]
result_submission["HPL"] = predict[:,1]
result_submission["MWS"] = predict[:,2]
result_submission.head()

,id,EAP,HPL,MWS
0,id02310,0.004352,0.000608,9.950402e-01
1,id24541,0.999940,0.000057,2.727059e-06
2,id00134,0.616386,0.383546,6.749213e-05
3,id27757,0.058671,0.941329,3.724144e-07
4,id04081,0.972442,0.016481,1.107696e-02


In [37]:
result_submission.to_csv('/home/emin/PycharmProjects/machine_learning/data/spooky author/submission.csv', index=False)